In [26]:
import pandas as pd
import numpy as np
import pickle as p

In [31]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=100, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [32]:
item_subset

[202,
 16019,
 3732,
 1230,
 3766,
 2589,
 2531,
 1195,
 3726,
 3743,
 5469,
 7114,
 7008,
 7575,
 2867,
 2701,
 7157,
 7434,
 1221,
 350,
 5501,
 2731,
 16018,
 2316,
 2256,
 4272,
 7128,
 204,
 2521,
 5581,
 16017,
 7965,
 3821,
 201,
 4978,
 3731,
 3773,
 3816,
 3768,
 2581,
 3772,
 3733,
 2257,
 3727,
 4725,
 7109,
 4270,
 102,
 3728,
 3045,
 4268,
 1502,
 3723,
 197,
 2571,
 7113,
 2501,
 3047,
 5068,
 1615,
 2551,
 2887,
 16003,
 7966,
 4273,
 3718,
 200,
 7967,
 7159,
 16002,
 5771,
 6750,
 4265,
 1231,
 3818,
 3736,
 22,
 3716,
 11,
 5069,
 2588,
 5074,
 7158,
 1004,
 5060,
 16001,
 2721,
 7080,
 3767,
 281,
 7108,
 7115,
 203,
 4977,
 196,
 31,
 1152,
 2736,
 1503,
 21]

### `/train`

In [33]:
import requests
import json

url = 'https://fillet.azurewebsites.net/train/'

data_subset.to_parquet('data_subset.parquet')
# data = data_subset.to_json()

files = {'data':open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True,
           'project_id': 4
           }

headers = {
           'Accept-Charset': 'UTF-8'
          }

# payload = json.dumps(payload)
r = requests.post(url, data=payload,  timeout=1200, files=files)

files['data'].close()

In [23]:
r.json()['result']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [34]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'11': 4.95,
 '21': 4.2,
 '22': 9.9,
 '31': 2.45,
 '102': 10.5,
 '196': 5.7,
 '197': 11.2,
 '200': 5.85,
 '201': 11.05,
 '202': 1.9,
 '203': 5.1,
 '204': 10.8,
 '281': 5.3,
 '350': 2.5,
 '1004': 3.75,
 '1152': 3.95,
 '1195': 3.45,
 '1221': 3.4,
 '1230': 3.5,
 '1231': 4.25,
 '1502': 3.55,
 '1503': 4.3,
 '1615': 2.85,
 '2256': 8.3,
 '2257': 11.6,
 '2316': 13.5,
 '2501': 2.6,
 '2521': 5.1,
 '2531': 6.4,
 '2551': 7.95,
 '2571': 6.15,
 '2581': 5.45,
 '2588': 6.7,
 '2589': 11.7,
 '2701': 5.95,
 '2721': 13.25,
 '2731': 7.8,
 '2736': 10.0,
 '2867': 5.5,
 '2887': 7.9,
 '3045': 2.0,
 '3047': 3.25,
 '3716': 6.15,
 '3718': 7.55,
 '3723': 11.4,
 '3726': 10.0,
 '3727': 10.7,
 '3728': 11.4,
 '3731': 10.85,
 '3732': 11.55,
 '3733': 12.25,
 '3736': 8.4,
 '3743': 10.95,
 '3766': 9.75,
 '3767': 10.45,
 '3768': 11.15,
 '3772': 9.6,
 '3773': 10.3,
 '3816': 8.7,
 '3818': 10.1,
 '3821': 10.25,
 '4265': 7.05,
 '4268': 11.9,
 '4270': 8.15,
 '4272': 12.45,
 '4273': 13.15,
 '4725': 8.6,
 '4977': 12.45,
 '4978': 

In [35]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 4}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)